In [1]:
import os
import json
import time
import random
import lxml
import requests
import json
import time
import sys
import re
import os
from requests.exceptions import RequestException
from bs4 import BeautifulSoup as bsoup
import random
import pandas as pd

In [2]:
from contextlib import closing
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException

In [3]:

delhi_phd = pd.read_html("http://www.cse.iitd.ac.in/index.php/2011-12-29-23-14-30/alumni/phd-alumni", header=0, parse_dates=['Year'])

In [ ]:
delhi_faculty=pd.read_html("http://www.cse.iitd.ac.in/index.php/2011-12-29-23-14-30/faculty",skiprows=[0,2])

In [5]:
def simple_get(url):
    try:
        with closing(requests.Session().get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    print(e)

In [12]:
delhi_phd[0].values[1:5]

array([['Neha Sengupta',
        'Algorithms For Massive And Dynamic Graph Data Management',
        Timestamp('2019-01-01 00:00:00'),
        'Research Associate at Inception Institute of Artificial Intelligence, Abu Dhabi UAE'],
       ['Suvam Patra',
        'Robust Scene Geometry Recovery From Ego-Cameras For Visual Navigation',
        Timestamp('2019-01-01 00:00:00'),
        'Research Engineer at Scape Technologies'],
       ['Ankit Anand',
        'Symmetry Aware Inference And Decision Making In Probabilistic Models',
        Timestamp('2019-01-01 00:00:00'),
        'Research Scientist at Google DeepMind Montreal'],
       ['Jatin Batra', 'Dynamic Programming For Scheduling Problems',
        Timestamp('2019-01-01 00:00:00'), 'Post Doc at CWI, Amsterdam']],
      dtype=object)

In [31]:
#url='http://eprint.iitd.ac.in/browse?type=title'
supervisor_all=[]
subject_all=[]
inst_all=[]
key_all=[]
name_all=[]
base_url='http://eprint.iitd.ac.in'
for t in delhi_phd[0]['Thesis Title'].values:
    thesis=t
    search_url="http://eprint.iitd.ac.in/discover?scope=2074%2F7856&&query={0}&submit=".format(thesis)
    search_item=simple_get(search_url)
    if search_item is not None:
        supervisor=[]
        subject=[]
        inst=[]
        key=[]
        name=[]
        search_item_soup=bsoup(search_item,'html.parser')
        handle=search_item_soup.find('div', attrs={'class':"col-sm-9 artifact-description"})
        if handle is not None:
            handle=handle.find('a',href=True)['href']
            url=base_url+handle
            time.sleep(random.randint(20,25))
            search_item2=simple_get(url)
            if search_item2 is not None:
                search_item2_soup=bsoup(search_item2,'html.parser')
                content=search_item2_soup.find_all("meta")
                for a in content:
                    if a.attrs.get('name')=="DC.contributor":
                        supervisor.append(a['content'])
                    elif a.attrs.get('name')=="DC.subject":
                        subject.append(a['content'])
                    elif a.attrs.get('name')=="citation_dissertation_institution":
                        inst.append(a['content'])
                    elif a.attrs.get('name')=="citation_keywords":
                        key.append(a['content'])
                    elif a.attrs.get('name')=="DC.creator":
                        name.append(a['content'])
                    else:
                        continue
    supervisor_all.append(supervisor)
    subject_all.append(subject)
    inst_all.append(inst)
    key_all.append(key)
    name_all.append(name)

In [22]:
delhi_phd[0]['Name'].values

array(['Neetu Jindal', 'Neha Sengupta', 'Suvam Patra', 'Ankit Anand',
       'Jatin Batra', 'Prathmesh Shashikant', 'Manjeet Dahiya',
       'Anup Kumar Bhattacharya', 'Brojeshwar Bhowmick', 'C. Sandeep',
       'Rahul Jain', 'Sonia Khetarpaul', 'Parul Shukla', 'Rajshekar K.',
       'Chinmay Narayan', 'Mona Gupta (Jain)', 'Eldhose Peter',
       'Shibhashis Guha', 'Nisha Jain', 'Swati Sharma', 'Yamuna Prasad',
       'Tarun Beri', 'Syamantak Das', 'Shivendra Prasad', 'Richa Sharma',
       'Rajesh Kumar Pal', 'Rajeshwari D', 'Pooja Aggarwal',
       'Arun Parakh', 'Anamitra Roy Choudhary', 'Amit Ruhela',
       'Vaibhav Jain', 'Prasenjit Chakraborty', 'Namita Sharma',
       'Mansureh Shahraki Moghaddam', 'Manoj D gupta', 'Ankur Narang',
       'L. Rajya Lakshmi', 'Gummidipudi Krishnaiah', 'Sankaran Anand',
       'Arindam Pal', 'Chetan Arora', 'Apurva Kumar',
       'Rudra Mohan Tripathy', 'Tanveer Afzal Faruquie', 'B.V.N.Silpa',
       'Ayesha Choudhary', 'Neeraj Goel', 'Smruti Padh

In [37]:
res=0
for a, b in zip(name_all,supervisor_all):
    if len(a) != 0 and len(b) != 0:
        res+=1
    else:
        continue

In [38]:
res

63

In [40]:
flat_list = [item for sublist in supervisor_all for item in sublist]

In [43]:
len(set(flat_list))

55

In [ ]:
iit_mum=pd.read_html('https://opac.library.iitb.ac.in/cgi-bin/koha/opac-search.pl?idx=kw&q=computer%20science&limit=mc-itype%3ATD&offset=0&sort_by=relevance_dsc')
iit_mum=iit_mum[0]
for i in range(10,1000,10):
    url="https://opac.library.iitb.ac.in/cgi-bin/koha/opac-search.pl?idx=kw&q=computer%20science&limit=mc-itype%3ATD&offset="+str(i)+"&sort_by=relevance_dsc"
    time.sleep(random.randint(10,15))
    tmp=pd.read_html(url)
    if len(tmp) != 0:
        iit_mum=pd.concat([iit_mum, tmp[0]])
    else:
        break

In [54]:
iit_mum1=iit_mum[2]

In [102]:
iit_mum2=iit_mum[2]

In [106]:
iit_mum2.to_csv('./college/iit_mum.csv', header=False, index=False)

In [ ]:
a=pd.read_html('https://opac.library.iitb.ac.in/cgi-bin/koha/opac-search.pl?idx=kw&
               q=computer%20science&limit=mc-itype%3ATD&offset=600&sort_by=relevance_dsc')

In [55]:
iit_mum1=iit_mum1.replace('No cover image available  ','',regex=True)

In [56]:
iit_mum1=iit_mum1.reset_index()

In [57]:
iit_mum1=iit_mum1.drop(columns=['index'])

In [59]:
iit_mum1[['title','other']]=iit_mum1[2].str.split(" by ",1,expand=True)

In [60]:
iit_mum1[['author','other']]=iit_mum1['other'].str.split('|',1,expand=True)

In [62]:
iit_mum1[['supervisor','other']]=iit_mum1['other'].str.split('Edition',1,expand=True)

In [64]:
iit_mum1[['year','other']]=iit_mum1['other'].str.split('Online',1,expand=True)

In [66]:
iit_mum1[['publisher','year']]=iit_mum1['year'].str.split(';',1,expand=True)

In [68]:
iit_mum1=iit_mum1.drop(columns=[2])

In [76]:
iit_mum1[['sup1','sup2','other1']]=iit_mum1['supervisor'].str.split('|',2,expand=True)

In [100]:
iit_mum1[['date']]=iit_mum1['supervisor'].str.extract('(\d+)')

In [86]:
iit_mum1.values[539]

array(['Shoreline change and coastal vulnerability considering projected climate (R)',
       None, 'Rajasree, B.R ',
       ' Deo, M.C [Supervisor] | Indian Institute of Technology Bombay. Department of Civil Engineering.Material type: Book; Format: print  ; Literary form: Not fiction  Publisher: Mumbai IIT 2018Dissertation note: Thesis Ph.D. Indian Institute of Technology Bombay. Department of Civil Engineering 2018 Availability: Items available for reference: [Call number: 043:551.465:681.342 Raj] (1). Lists:  Display List 27/05/2019-02/06/2019.  Add to cart',
       None, None, ' Deo, M.C [Supervisor] ',
       ' Indian Institute of Technology Bombay. Department of Civil Engineering.Material type: Book; Format: print  ; Literary form: Not fiction  Publisher: Mumbai IIT 2018Dissertation note: Thesis Ph.D. Indian Institute of Technology Bombay. Department of Civil Engineering 2018 Availability: Items available for reference: [Call number: 043:551.465:681.342 Raj] (1). Lists:  Display

In [101]:
iit_mum1=iit_mum1.drop(columns=[])

,title,other,author,supervisor,year,publisher,sup1,sup2,other1,date
0,Expert system for network topology determinati...,access: Click here to access online Availabil...,"Vidya Sagar, V.S.K.E","Khaparde, S.A. and Biswas, S.",IIT ; 1988,: Publisher: Bombay,"Khaparde, S.A. and Biswas, S.",None,None,NaN
1,Theme based planning in an uncertain environme...,access: Click here to access online Availabil...,"Khemani, Deepak","Ramakrishna, R.S.",IIT ; 1988,: Publisher: Bombay,"Ramakrishna, R.S.",None,None,NaN
2,Software development for discrete time nonline...,access: Click here to access online Availabil...,"Behera, Srinibas","Srisailam, M.C.",IIT ; 1989,: Bombay : I Publisher: Bombay,"Srisailam, M.C.",None,None,NaN
3,Sequential decision making in farm level water...,access: Click here to access online Availabil...,"Maadelat, M","Murthy, J.S.R.",IIT ; 1989,: Publisher: Bombay,"Murthy, J.S.R.",None,None,NaN
4,Analysis and design of vector quantizers (R),access: Click here to access online Availabil...,"Joshi, Rajan Laxman","Poonacha, P.G.",IIT ; 1990,: Bombay : I Publisher: Bombay,"Poonacha, P.G.",None,None,NaN
5,Development and evaluation of expert system fo...,access: Click here to access online Availabil...,"Upadhye, S.A","Subash Babu, A.",IIT ; 1990,: Bombay : I Publisher: Bombay,"Subash Babu, A.",None,None,NaN
6,Steady state security evaluation of power syst...,access: Click here to access online Availabil...,"Nandapurkar, U.P","Khaparde, S.A.",IIT ; 1990,": Bombay, II Publisher: Bombay","Khaparde, S.A.",None,None,NaN
7,Knowledge-based expert systems for thermodynam...,access: Click here to access online Availabil...,"Paranjape, Prakash K","Kudchadkar, A.P.",IIT ; 1989,: Bomaby : C Publisher: Bombay,"Kudchadkar, A.P.",None,None,NaN
8,Rule based expert system for image classificat...,access: Click here to access online Availabil...,"Borwankar, Mohan V","Sahasrabudhe, S.C. and Shevgaonkar, R.K.",IIT ; 1991,: Publisher: Bombay,"Sahasrabudhe, S.C. and Shevgaonkar, R.K.",None,None,NaN
9,Algorithms for training artificial neural netw...,access: Click here to access online Availabil...,"Krishna Ramesh, V.N.V","Poonacha, P.G.",IIT ; 1990,: Bombay : I Publisher: Bombay,"Poonacha, P.G.",None,None,NaN


In [5]:
iit_mum=pd.read_html('https://opac.library.iitb.ac.in/cgi-bin/koha/opac-search.pl?idx=kw&q=computer%20science&limit=mc-itype%3ATD&offset=0&sort_by=relevance_dsc')

In [7]:
iit_mum1=pd.read_html('https://opac.library.iitb.ac.in/cgi-bin/koha/opac-search.pl?idx=kw&q=computer%20science&limit=mc-itype%3ATD&offset=10&sort_by=relevance_dsc')

In [11]:
iit_mum1[0].columns

Int64Index([0, 1, 2], dtype='int64')

In [228]:
iitmm=iitmm[1].str.split('|',expand=True)

In [224]:
iitm.values[7]

'Knowledge-based expert systems for thermodynamic choices in chemical plant simulation (R) by Paranjape, Prakash K | Kudchadkar, A.P.Edition: Bomaby : C Publisher: Bombay ; IIT ; 1989Online access: Click here to access online Availability: Items available for reference: [Call number: 043:66.01:681.342Par] (1). Add to cart'

In [230]:
iitmm.values[0,1]

' Khaparde, S.A. and Biswas, S.Edition: Publisher: Bombay ; IIT ; 1988Online access: Click here to access online Availability: Items available for reference: [Call number: 043:621.316.2:681.342Vid] (1). Add to cart'

In [252]:
iitmm[['a','b']]=iitmm[1].str.split('Edition',expand=True)

In [258]:
iitmm[['c','d','e']]=iitmm['b'].str.split(';',2,expand=True)

In [263]:
iitmm[['y','ex']]=iitmm['e'].str.split('Online',1,expand=True)

In [267]:
s1 = pd.Series(['a', 'b'])
s2 = pd.Series(['c', 'd'])
s1=pd.concat([s1, s2])

In [268]:
s1

0    a
1    b
0    c
1    d
dtype: object

In [81]:
import re 
  
# initializing string  
test_string = "There are 22355 apples for 4 persons"
  
# printing original string  
print("The original string : " + test_string) 
  
# using re.findall() 
# getting numbers from string  
temp =re.findall(r'\d+', test_string 
res = list(map(int, temp)) 

The original string : There are 22355 apples for 4 persons


In [82]:
temp

['22355', '4']

In [7]:
print('Hans-Jörg Schek'.encode('ISO-8859-1'))

b'Hans-J\xf6rg Schek'
